In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D, Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers, applications
from keras.optimizers import SGD
from keras import backend as K
from keras.utils import to_categorical

IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 5

if K.image_data_format() == 'channels_first':
    input_shape = (3, IMAGE_SIZE, IMAGE_SIZE)
else:
    input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)

# get the pretrained neural network
pretrained_conv = applications.VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
print('Pretrained Model loaded.')

# pretrained_conv.summary()

for layer in pretrained_conv.layers[:-3]:
    layer.trainable = False

# Create the model
model = Sequential()

# Add the pretrained convolutional base model
model.add(pretrained_conv)

# First
model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-4),
              metrics=['categorical_accuracy'])
print('Model compiled.')

Using TensorFlow backend.


Pretrained Model loaded.
Model compiled.


In [2]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                802848    
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
_________________________________________________________________
activation_2 (Activation)    (None, 2)                

In [3]:
'''
Reference:
https://keras.io/preprocessing/image/
https://stackoverflow.com/questions/42443936/keras-split-train-test-set-when-using-imagedatagenerator
'''
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

IMAGE_DATA_DIR = 'data_face_training/'

# this is the augmentation configuration we will use for training and validation
image_datagen = ImageDataGenerator(
        rescale=1./IMAGE_SIZE,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = image_datagen.flow_from_directory(
        IMAGE_DATA_DIR,  # this is the target directory
        target_size=(IMAGE_SIZE, IMAGE_SIZE),  # all images will be resized to 150x150
        batch_size=BATCH_SIZE,
        color_mode='rgb',
        class_mode='categorical',
        shuffle=True,
        subset='training')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = image_datagen.flow_from_directory(
        IMAGE_DATA_DIR,
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        color_mode='rgb',
        class_mode='categorical',
        shuffle=True,
        subset='validation')

Found 2406 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [4]:
label_map = (train_generator.class_indices)
print(label_map)

{'happy': 0, 'unhappy': 1}


In [9]:
# Tensorboard reference: https://www.tensorflow.org/tensorboard/get_started
import datetime
import keras
%load_ext tensorboard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit_generator(
        train_generator,
        steps_per_epoch=100 // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=validation_generator,
        validation_steps=40 // BATCH_SIZE,
        callbacks=[tensorboard_callback])

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch 1/5
3/3 [==============================] - 46s 15s/step - loss: 0.3529 - categorical_accuracy: 0.8125 - val_loss: 0.4046 - val_categorical_accuracy: 0.8125
Epoch 2/5
3/3 [==============================] - 41s 14s/step - loss: 0.4025 - categorical_accuracy: 0.8750 - val_loss: 0.6721 - val_categorical_accuracy: 0.7812
Epoch 3/5
3/3 [==============================] - 41s 14s/step - loss: 0.3777 - categorical_accuracy: 0.8542 - val_loss: 0.3000 - val_categorical_accuracy: 0.9062
Epoch 4/5
3/3 [==============================] - 42s 14s/step - loss: 0.2914 - categorical_accuracy: 0.8958 - val_loss: 0.3433 - val_categorical_accuracy: 0.8438
Epoch 5/5
3/3 [==============================] - 41s 14s/step - loss: 0.4078 - categorical_accuracy: 0.8229 - val_loss: 0.2134 - val_categorical_accuracy: 0.9062


In [10]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
OUTPUT_MODEL_DIR = 'output_models/'
output_model = 'face-expression-model.h5'

model.save(OUTPUT_MODEL_DIR+output_model)